In [1]:
import olympe
import os
import subprocess
import tempfile
from olympe.media import (
    media_created,
    resource_created,
    media_removed,
    resource_removed,
    resource_downloaded,
    indexing_state,
    delete_media,
    download_media,
    download_media_thumbnail,
    MediaEvent,
)
from olympe.messages.camera import (
    set_camera_mode,
    set_photo_mode,
    take_photo,
    photo_progress,
)
from io import StringIO, BytesIO

# from logging import getLogger

# olympe.log.update_config(
#     {
#         "loggers": {
#             "olympe": {"level": "INFO", "handlers": ["console"]},
#             "urllib3": {"level": "DEBUG", "handlers": ["console"]},
#             __name__: {"level": "DEBUG", "handlers": ["console"]},
#         }
#     }
# )

# logger = getLogger(__name__)

DRONE_IP = os.environ.get("DRONE_IP", "10.202.0.1")
DRONE_MEDIA_PORT = os.environ.get("DRONE_MEDIA_PORT", "80")
img = None

In [2]:
class MediaEventListener(olympe.EventListener):
    def __init__(self, media):
        super().__init__(media, timeout=60)
        self._media = media
#         self._media_id = []
#         self._downloaded_resources = []
#         self.remote_resource_count = 0
#         self.local_resource_count = 0

    @olympe.listen_event(media_created())
    def onMediaCreated(self, event, scheduler):
        self._media(download_media(event.media_id))

#     @olympe.listen_event(resource_created())
#     def onResourceCreated(self, event, scheduler):
#         logger.info("resource_created {}".format(event.resource_id))

#     @olympe.listen_event(media_removed())
#     def onMediaRemoved(self, event, scheduler):
#         logger.info("media_removed {}".format(event.media_id))

#     @olympe.listen_event(resource_removed())
#     def onResourceRemoved(self, event, scheduler):
#         logger.info("resource_removed {}".format(event.resource_id))

    @olympe.listen_event(resource_downloaded())
    def onResourceDownloaded(self, event, scheduler):
        if event.is_thumbnail:
            return
        print(type(event))
        global img
        with open(event.download_path, "rb") as image_file:
            image_data = image_file.read()
            img = Image.open(BytesIO(image_data))

# #         print(type(image_file))
# #         print(type(image_data))
#         global img

#         logger.info(
#             "resource_downloaded {} {}".format(
#                 event.resource_id,
#                 event.data["download_path"],
#             )
#         )
#         self._downloaded_resources.append(
#             self._media.resource_info(resource_id=event.resource_id)
#         )

    @olympe.listen_event()
    def default(self, event, scheduler):
        if isinstance(event, MediaEvent):
            logger.info(event)

    def unsubscribe(self):
        self._media.wait_for_pending_downloads()
        # Sanity check 1/2: md5 checksum
        # The integrity check has already been performed by Olympe
        # For this example the following step demonstrate how to perform the media
        # integrity check afterward using the "md5summ --check *.md5" command.
        for resource in self._downloaded_resources:
            check = subprocess.run(
                ["md5sum", "--check", "{}.md5".format(resource.download_path)],
                stdout=subprocess.PIPE,
                cwd=os.path.dirname(resource.download_path),
            )
            stdout = check.stdout.decode().strip()
            if check.returncode == 0:
                logger.info("Integrity check: " + stdout)
            else:
                logger.error("Integrity check: " + stdout)
                super().unsubscribe()
                return

        # Sanity check 2/2: local downloaded resources equals the number of remote resources
        self.remote_resource_count = sum(
            map(
                lambda id_: len(self._media.resource_info(media_id=id_)), self._media_id
            )
        )
        self.local_resource_count = len(self._downloaded_resources)
        if self.local_resource_count != self.remote_resource_count:
            logger.error(
                "Downloaded {} resources instead of {}".format(
                    self.local_resource_count,
                    self.remote_resource_count,
                )
            )
            super().unsubscribe()
            return

        # OK then, we can now safely delete the remote media
        for media_id in self._media_id:
            delete = delete_media(media_id, _timeout=10)
            if not self._media(delete).wait().success():
                logger.error(
                    "Failed to delete media {} {}".format(media_id, delete.explain())
                )
        super().unsubscribe()

In [3]:
def setup_photo_burst_mode(drone):
    drone(set_camera_mode(cam_id=0, value="photo")).wait()
    # For the file_format: jpeg is the only available option
    # dng is not supported in burst mode
    drone(
        set_photo_mode(
            cam_id=0,
            mode="single",
            format="rectilinear",
            file_format="jpeg",
            burst="burst_14_over_1s",
            # the following parameters are ignored in photo burst mode but we
            # must provide valid values for them anyway
            bracketing="preset_1ev",
            capture_interval=5.0,
        )
    ).wait()

In [4]:
def main(drone, media=None):
    setup_photo_burst_mode(drone)
    if media is None:
        assert drone.media_autoconnect
        media = drone.media
    media.download_dir = tempfile.mkdtemp(prefix="olympe_media_example_")
    media.integrity_check = True
#     logger.info("waiting for media resources indexing...")
    if not media(indexing_state(state="indexed")).wait(_timeout=60).success():
#         logger.error("Media indexing timed out")
        return
#     logger.info("media resources indexed")
    with MediaEventListener(media) as media_listener:
        photo_saved = drone(photo_progress(result="photo_saved", _policy="wait"))
        drone(take_photo(cam_id=0)).wait()
        if not photo_saved.wait(_timeout=30).success():
#             logger.error("Photo not saved: {}".format(photo_saved.explain()))
            print("Photo not saved: {}".format(photo_saved.explain()))
            return
#     assert (
#         media_listener.remote_resource_count == 14
#     ), "remote resource count = {}".format(media_listener.remote_resource_count)
#     assert (
#         media_listener.local_resource_count == 14
#     ), "local resource count = {}".format(media_listener.local_resource_count)

In [5]:
def test_media():
    with olympe.Drone(
        DRONE_IP,
        media_autoconnect=False,
        media_port=DRONE_MEDIA_PORT,
        name="example_media_standalone",
    ) as drone:
        assert drone.connect()
        media = olympe.Media(
            f"{DRONE_IP}:{DRONE_MEDIA_PORT}", name="example_media_standalone"
        )
        assert media.connect()
        main(drone, media)
        assert media.shutdown()
        assert drone.disconnect()

In [6]:
test_media()

2022-02-06 14:53:43,438 [INFO] 	olympe.example_media_standalone.backend - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:43,456 [INFO] 	olympe.example_media_standalone.backend - _do_create - device callbacks have been added to arsdk_ctrl
2022-02-06 14:53:43,483 [INFO] 	olympe.example_media_standalone.scheduler - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:43,518 [INFO] 	olympe.module_loader - add_package_root - ModuleLoader 'olympe.airsdk.example_media_standalone' root package has been registered
2022-02-06 14:53:43,542 [INFO] 	olympe.example_media_standalone.backend - _device_added_cb - DiscoveryNet: New device has been detected: 'ANAFI-0000000'
2022-02-06 14:53:43,548 [INFO] 	olympe.example_media_standalone.drone - _connecting_cb - Connecting to device: ANAFI-0000000
2022-02-06 14:53:43,552 [INFO] 	olympe.example_media_standalone.drone - _connect_impl - Connection in progress...
2022-02-06 14:53:43,554 [INFO] 	olympe.example_media_standalone.drone - _connected_cb

2022-02-06 14:53:43,885 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - ardrone3.PilotingState.VibrationLevelChanged(state='ok')
2022-02-06 14:53:43,887 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - common.MavlinkState.MavlinkFilePlayingStateChanged(state='stopped', filepath='', type='flightPlan')
2022-02-06 14:53:43,889 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - auto_follow.config(distance=0.0, elevation=0.0, azimuth=0.0)
2022-02-06 14:53:43,890 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - auto_follow.state(mode='none', behavior='idle')
2022-02-06 14:53:43,893 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - auto_look_at.state(mode='none', behavior='idle')
2022-02-06 14:53:43,895 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - follow_me.state(mode='none', behavior='idle', animation='none', animation_available='')
2022-02-06 14:53:43,898 [INFO] 	olympe.example_media_standalone.drone - _re

2022-02-06 14:53:43,965 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - ardrone3.MediaRecordState.VideoStateChangedV2(state='notAvailable', error='ok')
2022-02-06 14:53:43,966 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_state(cam_id=0, available='not_available', state='inactive', list_flags='')
2022-02-06 14:53:43,969 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.alignment_offsets(cam_id=0, min_bound_yaw=0.0, max_bound_yaw=0.0, current_yaw=0.0, min_bound_pitch=0.0, max_bound_pitch=0.0, current_pitch=0.0, min_bound_roll=0.0, max_bound_roll=0.0, current_roll=0.0, list_flags='')
2022-02-06 14:53:43,971 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.antiflicker_mode(mode='mode_50hz', value='mode_50hz')
2022-02-06 14:53:43,973 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.camera_states(active_cameras=1)
2022-02-06 14:53:43,975 [INFO] 	olympe.example_media_standalone.drone - 

2022-02-06 14:53:44,213 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - gimbal.gimbal_capabilities(gimbal_id=0, model='main', axes='pitch|roll', list_flags='')
2022-02-06 14:53:44,215 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.antiflicker_capabilities(supported_modes='off|mode_50hz|mode_60hz')
2022-02-06 14:53:44,218 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - leds.capabilities(supported_capabilities='')
2022-02-06 14:53:44,219 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - precise_home.capabilities(modes='disabled|standard')
2022-02-06 14:53:44,226 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - stereo_vision_sensor.capabilities(sensor_id=0, model='main', supported_features='')
2022-02-06 14:53:44,236 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - piloting_style.capabilities(styles='standard|camera_operated')
2022-02-06 14:53:44,244 [INFO] 	olympe.example_media_standalone.drone 

2022-02-06 14:53:44,567 [INFO] 	olympe.example_media_standalone.pdraw - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:44,581 [INFO] 	olympe.example_media_standalone.pdraw - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:44,594 [INFO] 	olympe.example_media_standalone.media - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:44,599 [INFO] 	olympe.example_media_standalone.scheduler - _create_pomp_loop - Creating pomp loop
2022-02-06 14:53:44,640 [ERROR] 	olympe.example_media_standalone.media - _get_all_media - 541 Server Error: Media Not Yet Indexed for url: http://10.202.0.1:80/api/v1/media/medias
2022-02-06 14:53:44,644 [WARNING] 	olympe.example_media_standalone.media - _websocket_connect_cb - Media are not yet indexed
2022-02-06 14:53:44,661 [INFO] 	olympe.example_media_standalone.drone - _send_command_impl - camera.set_camera_mode(cam_id=0, value='photo') has been sent to the device
2022-02-06 14:53:44,663 [INFO] 	olympe.example_media_standalone.drone - _recv_

2022-02-06 14:53:45,065 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.recording_capabilities(id=3, recording_modes='slow_motion|high_framerate', resolutions='res_720p', framerates='', hdr='not_supported', list_flags='Last')
2022-02-06 14:53:45,068 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_capabilities(id=0, photo_modes='single|time_lapse|gps_lapse', photo_formats='full_frame', photo_file_formats='dng_jpeg', hdr='supported', list_flags='First')
2022-02-06 14:53:45,075 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_capabilities(id=1, photo_modes='single|burst|time_lapse|gps_lapse', photo_formats='full_frame', photo_file_formats='jpeg', hdr='supported', list_flags='')
2022-02-06 14:53:45,078 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_capabilities(id=2, photo_modes='single|burst|time_lapse|gps_lapse', photo_formats='rectilinear', photo_file_formats='jpeg', hdr='supported'

2022-02-06 14:53:47,301 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - follow_me.mode_info(mode='geographic', missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|image_detection|target_good_speed|drone_close_enough', list_flags='')
2022-02-06 14:53:47,304 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - follow_me.mode_info(mode='relative', missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|image_detection|target_good_speed|drone_close_enough', list_flags='')
2022-02-06 14:53:47,306 [I

2022-02-06 14:53:52,259 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_progress(cam_id=0, result='photo_saved', photo_count=0, media_id='10000001', list_flags='')
2022-02-06 14:53:52,263 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - camera.photo_state(cam_id=0, available='available', state='inactive', list_flags='')
2022-02-06 14:53:52,539 [INFO] 	olympe.example_media_standalone.drone - _recv_cmd_cb - ardrone3.GPSSettingsState.GeofenceCenterChanged(latitude=48.87890000000001, longitude=2.3677785)
2022-02-06 14:53:52,552 [INFO] 	olympe.example_media_standalone.media - _websocket_event_cb - media_event(name=media_created, media_id=10000001, photo_mode='SINGLE')
2022-02-06 14:53:52,648 [INFO] 	olympe.example_media_standalone.media - _write_chunk - Download 100000010001.JPG  100% done
2022-02-06 14:53:52,656 [INFO] 	olympe.example_media_standalone.pdraw - _destroy_pomp_loop - Pomp loop has been destroyed: Thread-7
2022-02-06 14:53:52,663 [ERROR

<class 'olympe.media.MediaEvent'>


AttributeError: 'MediaEventListener' object has no attribute '_downloaded_resources'

In [7]:
type(img)

NoneType

2022-02-06 14:55:57,367 [INFO] 	olympe.example_media_standalone.media - wrapper - Websocket already closed
2022-02-06 14:55:57,369 [ERROR] 	ulog - pomp - epoll_ctl(fd=95) err=9(Bad file descriptor)
2022-02-06 14:55:57,370 [ERROR] 	ulog - pomp - epoll_ctl op=2 cb=0x7f66e9b098d0 userdata=0x7f669b30d710
